In [1]:
import VisumPy.helpers as h
import pandas as pd


In [2]:
Visum = h.CreateVisum(18)

In [3]:
#This version file is FOR TESTING ONLY - update to new model version when ready
versionFilePath = "D:/dvrpc_shared/BikeLTS/Phase3/NetworkUpdates/TIM23_2020_forAnalysis_072920.ver"
Visum.LoadVersion(versionFilePath)

In [4]:
#lookup table for lane, speed
#second residential line modified to include speed up to 65 to capture everything
road_index = [
    [(0, 0),   (0,  999)],
    [(-2, -2), (0,  25 )], 
    [(-2, -2), (26, 36 )], 
    [(1, 3),   (0,  25 )],
    [(4, 5),   (0,  25 )],
    [(1, 3),   (26, 34 )],
    [(6, 999), (0,  25 )],
    [(4, 5),   (26, 34 )],
    [(6, 999), (26, 34 )],
    [(1, 3),   (35, 999)],
    [(4, 5),   (35, 999)],
    [(6, 999), (35, 999)],
]

residential_index = [
    [(0, 0),   (0,  999)],
    [(1, 2),   (0,  25 )], 
    [(1, 2),   (26, 65 )], 
]


#create lookup for bike facility type
#numbers re-ordered to act as crosswalk between model bike fac codes and those in the reduction factor table
bikeFac_index = [0, 5, 1, 2, 3, 4, 6, 9]

#from LTS table
#row 0 is filler for roads with no lanes
#column 7 is filler for roads with bike fac  =  (opposite direction of a one way street)
StressLevels = [
    [-1, -1, -1, -1, -1, -1, -1, -1],
    [ 1,  1,  1,  1,  1,  1,  1, -2],
    [ 2,  2,  2,  1,  1,  1,  1, -2],
    [ 2,  2,  2,  1,  1,  1,  1, -2],
    [ 3,  3,  3,  2,  2,  1,  1, -2],
    [ 3,  3,  3,  2,  2,  1,  1, -2],
    [ 4,  4,  4,  3,  2,  2,  1, -2],
    [ 4,  4,  4,  3,  2,  2,  1, -2],
    [ 4,  4,  4,  3,  2,  2,  1, -2],
    [ 4,  4,  4,  3,  3,  2,  1, -2],
    [ 4,  4,  4,  3,  3,  2,  1, -2],
    [ 4,  4,  4,  4,  3,  3,  1, -2],
]

In [5]:
#function to identify the row of Figure 1 in paper that the record falls in based on the total number of lanes and the speed
def findRowIndex(lanes, spd, link_type):
    if link_type in (72, 79) and lanes in (1, 2):
        for i, ((minlanes,maxlanes), (lowerspd, upperspd)) in enumerate(residential_index):
            if (minlanes <= lanes and lanes <= maxlanes) and (lowerspd <= spd and spd <= upperspd):
                return i
    else:
        for i, ((minlanes,maxlanes), (lowerspd, upperspd)) in enumerate(road_index):
            if (minlanes <= lanes and lanes <= maxlanes) and (lowerspd <= spd and spd <= upperspd):
                return i

In [6]:
#function to idenfity the colum of Figure 1 based on the bicycle facility
def bikeFacLookup(fac_code):
    for i, (facility) in enumerate(bikeFac_index):
        if facility == fac_code:
            return i

In [7]:
def make_attribute_list(att_name):
    #get attributes from Visum
    map = h.GetMulti(Visum.Net.Links, att_name)
    #convert map object type to list
    att_name = list(map)
    return att_name
    
FromNode = make_attribute_list("FromNodeNo")
ToNode   = make_attribute_list("ToNodeNo")
Length   = make_attribute_list("Length")
TotLanes = make_attribute_list("TotNumLanes") #this may need to be calculated first or on the fly with a new version
BikeFac  = make_attribute_list("Bike_Facility") #this may have a dif name
Speed    = make_attribute_list("SPEED_LTS") #this will need a dif name
LinkType = make_attribute_list("TypeNo")
WKTPoly  = make_attribute_list("WKTPoly")
Slope    = make_attribute_list("SLOPE_PERC") #check name of this

In [8]:
LinkStress = [0]* len(FromNode)

for i in range(0, len(FromNode)):
	x = bikeFacLookup(BikeFac[i])
	y = findRowIndex(TotLanes[i], Speed[i], LinkType[i])
	LinkStress[i] = StressLevels[y][x]

In [10]:
import geopandas as gpd

OSError: Could not find lib geos_c.dll or load any of its variants [].

In [32]:
WKTPoly.crs

AttributeError: 'list' object has no attribute 'crs'

In [31]:
#combine attributes into dataframe
df = pd.DataFrame(
    {'FromNode': FromNode,
    'ToNode'   : ToNode,
    'Length'   : Length,
    'TotLanes' : TotLanes,
    'BikeFac'  : BikeFac,
    'Speed'    : Speed,
    'LinkType' : LinkType,
    'Slope'    : Slope,
    'Geom'     : WKTPoly
    }
)
gs = gpd.GeoSeries.from_wkt(df['Geom'])
gdf = gpd.GeoDataFrame(df, geometry = gs)


#import into DB

NameError: name 'gpd' is not defined